<a target="_blank" href="https://colab.research.google.com/github/Tensor-Reloaded/Neural-Networks-Template-2024/blob/main/Lab01/Assignment1.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


# **Assignment 1 (10 points)**


## **Solving a linear system in python**


In this homework, you will familiarize yourself with key linear algebra con-
cepts and Python programming by solving a system of linear equations. You
will explore multiple methods for solving such systems, including Cramer’s rule
and matrix inversion. By the end of this assignment, you will have a good un-
derstanding of how to represent and manipulate matrices and vectors in Python.


We begin with the following system of 3 linear equations with 3 unknowns:
$$ 2x + 3y - z = 5 $$
$$ x - y + 4z = 6 $$
$$ 3x + y + 2z = 7 $$

This system can be vectorized in the following form:
$$ A \cdot X = B $$
where:

$$
A = \begin{bmatrix}
2 & 3 & -1 \\
1 & -1 & 4 \\
3 & 1 & 2
\end{bmatrix}, \quad
X = \begin{bmatrix}
x \\
y \\
z
\end{bmatrix}, \quad
B = \begin{bmatrix}
5 \\
6 \\
7
\end{bmatrix}
$$

**Considerations**

- do not use any linear algebra framework such as $numpy$
- use python lists as data structures for matrices and vectors
- experiment with other values for the coefficients and free terms


### **1. Parsing the System of Equations (1 point)**


The first task is to implement a Python script that reads a system of linear equations from a text file and parses it into a matrix $A$ and a vector $B$. You will use the input format described below to extract the coefficients for $A$ and $B$.

**Input File Format**

```text
2x + 3y - z = 5
x - y + 4z = 6
3x + y + 2z = 7
```

Note that the coefficients are always in the order x, y and z and the terms are always space separated


In [21]:
import pathlib


def get_signs(row: list[str]) -> list[str]:
    signs = ["+"]

    for token in row:
        if token == "+" or token == "-":
            signs.append(token)

    return signs


def get_coefficients(row: list[str]) -> list[float]:
    coefficients = []
    coefficient_str = ""

    for token in row:
        if "x" not in token and "y" not in token and "z" not in token:
            continue

        for chr in token:
            if chr.isdigit() or chr == ".":
                coefficient_str += chr
        
        if(coefficient_str == ""):
            coefficient_str = "1"

        coefficients.append(float(coefficient_str))
        coefficient_str = ""

    return coefficients

def combine_signs_and_coefficients(signs: list[str], coefficients: list[float]) -> list[float]:
    result = []

    for i in range(len(signs)):
        if signs[i] == "+":
            result.append(coefficients[i])
        else:
            result.append(-coefficients[i])

    return result

def get_result(row: list[str]) -> float:
    return float(row[-1])


def load_system(path: pathlib.Path) -> tuple[list[list[float]], list[float]]:
    A = []
    B = []

    with open(path, "r") as file:
        file_content = file.readlines()

    for file_row in file_content:
        file_row_elements = file_row.split(" ")
        signs = get_signs(file_row_elements)
        coefficients = get_coefficients(file_row_elements)
        result = get_result(file_row_elements)
        coefficients_with_signs = combine_signs_and_coefficients(signs, coefficients)
        
        A.append(coefficients_with_signs)
        B.append(result)
        
        
    return A,B


A, B = load_system(pathlib.Path("input.txt"))
print(f"{A=} {B=}")

A=[[2.0, 3.0, -1.0], [1.0, -1.0, 4.0], [3.0, 1.0, 2.0]] B=[5.0, 6.0, 7.0]


### **2. Matrix and Vector Operations (5 points)**


Once you have successfully parsed the matrix and vector, complete the following exercises to manipulate and understand basic matrix and vector operations. Write Python functions for each of these tasks:


#### 2.1. Determinant


Write a function to compute the determinant of matrix $A$. Recall one of the formulae for the determinant of a $3x3$ matrix:
$$ \text{det}(A) = a*{11}(a*{22}a*{33} - a*{23}a*{32}) - a*{12}(a*{21}a*{33} - a*{23}a*{31}) + a*{13}(a*{21}a*{32} - a*{22}a\_{31}) $$


In [22]:
def determinant(matrix: list[list[float]]) -> float:
    if len(matrix) == 2:
        return matrix[0][0] * matrix[1][1] - matrix[0][1] * matrix[1][0]
    elif len(matrix) == 3:
        a11, a12, a13 = matrix[0]
        a21, a22, a23 = matrix[1]
        a31, a32, a33 = matrix[2]

        return (
            a11 * (a22 * a33 - a23 * a32)
            - a12 * (a21 * a33 - a23 * a31)
            + a13 * (a21 * a32 - a22 * a31)
        )
    else:
        raise ValueError("Only 2x2 and 3x3 matrices are supported")


print(f"{determinant(A)=}")

determinant(A)=14.0


#### 2.2. Trace


Compute the sum of the elements along the main diagonal of matrix $A$. For a matrix $A$, this is:
$$ \text{Trace}(A) = a*{11} + a*{22} + a\_{33} $$


In [23]:
def trace(matrix: list[list[float]]) -> float:
    a11 = matrix[0][0]
    a22 = matrix[1][1]
    a33 = matrix[2][2]

    tr = a11 + a22 + a33

    return tr

print(f"{trace(A)=}")

trace(A)=3.0


#### 2.3. Vector norm


Compute the Euclidean norm of vector $B$, which is:
$$ ||B|| = \sqrt{b_1^2 + b_2^2 + b_3^2} $$


In [24]:
import math

def norm(vector: list[float]) -> float:
    sum_of_squares = 0
    
    for x in vector:
        sum_of_squares += x ** 2
    
    euclidean_norm = math.sqrt(sum_of_squares)
    
    return euclidean_norm

print(f"{norm(B)=}")

norm(B)=10.488088481701515


#### 2.4. Transpose of matrix


Write a function to compute the transpose of matrix $A$. The transpose of a matrix $A$ is obtained by swapping its rows and columns.


In [25]:
def transpose(matrix: list[list[float]]) -> list[list[float]]:
    transposed_matrix = [
        [matrix[0][0], matrix[1][0], matrix[2][0]],
        [matrix[0][1], matrix[1][1], matrix[2][1]],
        [matrix[0][2], matrix[1][2], matrix[2][2]]
    ]
    return transposed_matrix

print(f"{transpose(A)=}")

transpose(A)=[[2.0, 1.0, 3.0], [3.0, -1.0, 1.0], [-1.0, 4.0, 2.0]]


#### 2.5. Matrix-vector multiplication


Write a function that multiplies matrix $A$ with vector $B$.


In [26]:
def multiply(matrix: list[list[float]], vector: list[float]) -> list[float]:
    result = [0.0, 0.0, 0.0]
    
    for i in range(3):
        result[i] = matrix[i][0] * vector[0] + matrix[i][1] * vector[1] + matrix[i][2] * vector[2]
    
    return result


print(f"{multiply(A, B)=}")

multiply(A, B)=[21.0, 27.0, 35.0]


### **3. Solving using Cramer's Rule (1 point)**


Now that you have explored basic matrix operations, solve the system of linear equations using Cramer's rule.

**Cramer's Rule:**

Cramer's rule allows you to solve for each unknown $x$, $y$, and $z$ using determinants. For example:
$$ x = \frac{\text{det}(A_x)}{\text{det}(A)}, \quad y = \frac{\text{det}(A_y)}{\text{det}(A)}, \quad z = \frac{\text{det}(A_z)}{\text{det}(A)} $$
where $A_x$, $A_y$, and $A_z$ are matrices formed by replacing the respective column of matrix $A$ with vector $B$.


In [27]:
def solve_cramer(matrix: list[list[float]], vector: list[float]) -> list[float]:
    det_A = determinant(matrix)
    
    matrix1 = [
        [vector[0], matrix[0][1], matrix[0][2]],
        [vector[1], matrix[1][1], matrix[1][2]],
        [vector[2], matrix[2][1], matrix[2][2]]
    ]
    matrix2 = [
        [matrix[0][0], vector[0], matrix[0][2]],
        [matrix[1][0], vector[1], matrix[1][2]],
        [matrix[2][0], vector[2], matrix[2][2]]
    ]
    matrix3 = [
        [matrix[0][0], matrix[0][1], vector[0]],
        [matrix[1][0], matrix[1][1], vector[1]],
        [matrix[2][0], matrix[2][1], vector[2]]
    ]

    det_Ax = determinant(matrix1)
    det_Ay = determinant(matrix2)
    det_Az = determinant(matrix3)

    x = det_Ax / det_A
    y = det_Ay / det_A
    z = det_Az / det_A

    return [x, y, z]


print(f"{solve_cramer(A, B)=}")

solve_cramer(A, B)=[0.35714285714285715, 2.0714285714285716, 1.9285714285714286]


### **4. Solving using Inversion (3 points)**


Finally, solve the system by computing the inverse of matrix $A$ and multiplying it by vector $B$.
$$ A \cdot X = B \rightarrow X = A^{-1} \cdot B $$
**Adjugate Method for Matrix Inversion:**

To find the inverse of matrix $ A $, you can use the adjugate method:
$$ A^{-1} = \frac{1}{\text{det}(A)} \times \text{adj}(A) $$
where $\text{adj}(A)$ is the adjugate (or adjoint) matrix, which is the transpose of the cofactor matrix of $ A $.

**Cofactor Matrix:**

The cofactor matrix is a matrix where each element is replaced by its cofactor. The cofactor of an element $a_{ij}$ is given by:
$$ (-1)^{i+j} \times \text{det}(M*{ij}) $$
where $M*{ij}$ is the minor of element $a_{ij}$, which is the matrix obtained by removing the $i$-th row and $j$-th column from matrix $A$.


In [29]:
def minor(matrix: list[list[float]], i: int, j: int) -> list[list[float]]:
    minor_matrix = []
    for row_index, row in enumerate(matrix):
        if row_index != i:
            new_row = []
            for col_index, element in enumerate(row):
                if col_index != j:
                    new_row.append(element)
            minor_matrix.append(new_row)
    return minor_matrix


def cofactor(matrix: list[list[float]]) -> list[list[float]]:
    cofactors = []
    for i in range(3):
        cofactor_row = []
        for j in range(3):
            minor_matrix = minor(matrix, i, j)
            cofactor_value = ((-1) ** (i + j)) * determinant(minor_matrix)
            cofactor_row.append(cofactor_value)
        cofactors.append(cofactor_row)
    return cofactors


def adjoint(matrix: list[list[float]]) -> list[list[float]]:
    cofactors = cofactor(matrix)
    adj = []
    for i in range(3):
        adj_row = []
        for j in range(3):
            adj_row.append(cofactors[j][i])
        adj.append(adj_row)
    return adj


def solve(matrix: list[list[float]], vector: list[float]) -> list[float]:
    det_A = determinant(matrix)
    
    adj_A = adjoint(matrix)
    
    result = [0.0, 0.0, 0.0]
    for i in range(3):
        sum_product = 0.0
        for j in range(3):
            sum_product += adj_A[i][j] * vector[j]
        result[i] = sum_product / det_A
    return result


print(f"{solve(A, B)=}")

solve(A, B)=[0.35714285714285715, 2.0714285714285716, 1.9285714285714286]
